In [70]:
import pandas as pd
import pandas_ta as ta
import matplotlib.pyplot as plt
import numpy as np
import scikitplot as skplot
import csv
import datetime as dt
import pandas_datareader.data as web
import os,sys,csv,time
import re
import urllib.request as urlopen
from bs4 import BeautifulSoup 
import os
import time
import requests
from sec_api import QueryApi

In [11]:
years = [1995 + x for x in range(0,26)]
quarters = ['QTR1', 'QTR2', 'QTR3', 'QTR4']

In [17]:
orgpath = 'C:\\Users\\whill\\OneDrive\\Desktop\\MGT of Fin Institutions\\Hills_A12\\Datafiles'
cfolds = os.listdir(orgpath)

## Save and convert master.idx files to directory, run once

In [21]:
for y in years:
    if str(y) not in cfolds:
        os.mkdir('\\'.join([orgpath, str(y)]))
    cfiles = os.listdir('\\'.join([orgpath,str(y)]))
    for q in quarters:
        newfile = f'master_{y}_{q}.txt'
        newfilepath = '\\'.join([orgpath,str(y),newfile])
        if newfile in cfiles:
            print(f'File {y}, {q} is already downloaded')
            continue
        idxurl = f'https://www.sec.gov/Archives/edgar/full-index/{y}/{q}/master.idx'
        hdr = {'User-Agent':'Mozilla/5.0'}
        req = requests.get(idxurl, stream = True, headers = hdr)
        with open(newfilepath, 'wb') as f:
            for content in req.iter_content(chunk_size = 10240):
                f.write(content)
        time.sleep(1)

## Read in files, dataframe, randomize, save to csv

In [52]:
count = 0
for y in years:
    yearfolder = '\\'.join([orgpath,str(y)])
    for q in quarters:
        readdf = pd.read_csv('\\'.join([yearfolder,f'master_{y}_{q}.txt']), skiprows=[0,1,2,3,4,5,6,7,8,10], sep = '|')
        readdf['Year'] = pd.to_datetime(readdf['Date Filed'], format = '%Y-%m-%d').dt.year
        readdf = readdf[(readdf['Form Type'] == '8-K')]
        readdf['Quarter'] = f'{q}'
        randdf = readdf.groupby(by='Year').sample(n=50, replace = True, random_state = 1).reset_index(drop=True)
        randdf = randdf[['CIK', 'Company Name', 'Form Type', 'Date Filed','Year', 'Quarter', 'Filename']]
        if count == 0:
            copyranddf = randdf.copy()
            count +=1
        else:
            copyranddf = pd.concat([copyranddf, randdf])
copyranddf.to_csv('8-Krandomsample.csv', sep = ',', header = copyranddf.columns)

In [56]:
rbcsv = pd.read_csv('8-Krandomsample.csv')
rbcsv.drop(['Unnamed: 0'], axis = 1, inplace = True)
rbcsv.head()

,CIK,Company Name,Form Type,Date Filed,Year,Quarter,Filename
0,877930,MESA INC,8-K,2/17/1995,1995,QTR1,edgar/data/877930/0000877930-95-000002.txt
1,314459,CARLYLE REAL ESTATE LTD PARTNERSHIP X,8-K,3/29/1995,1995,QTR1,edgar/data/314459/0000892626-95-000034.txt
2,90811,AMERICAN STORES CO /NEW/,8-K,3/10/1995,1995,QTR1,edgar/data/90811/0000090811-95-000001.txt
3,806641,SCA TAX EXEMPT FUND LIMITED PARTNERSHIP,8-K,3/22/1995,1995,QTR1,edgar/data/806641/0000806641-95-000005.txt
4,73902,OGDEN CORP,8-K,1/3/1995,1995,QTR1,edgar/data/73902/0000073902-95-000002.txt


## Run once to make directories

In [58]:
#for y in years:
    #os.mkdir('\\'.join([orgpath, str(y), '8_K_files']))

## Save all the 8-Ks

In [65]:
for i in range(len(rbcsv['CIK'])):
        newfile = f'{rbcsv.CIK.iloc[i]}_{rbcsv.Year.iloc[i]}_{rbcsv.Quarter.iloc[i]}_8K.txt'
        filepath = '\\'.join([orgpath,f'{rbcsv.Year.iloc[i]}','8_K_files',newfile])
        url = f'https://www.sec.gov/Archives/{rbcsv.Filename.iloc[i]}'
        hdr = {'User-Agent':'Mozilla/5.0'}
        req = requests.get(url, stream = True, headers = hdr)
        with open(filepath, 'wb') as f:
            for content in req.iter_content(chunk_size = 10240):
                f.write(content)
        time.sleep(1)